In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)

# Load Data

In [ ]:
cd8 <- readRDS(file = '../Seurat_Objects/cd8_subset.rds')

In [ ]:
cd8

In [ ]:
table(cd8$pbmc_sample_id)

In [ ]:
cd8.batch <- SplitObject(cd8, split.by = 'batch_id')

In [ ]:
cd8.batch <- lapply(cd8.batch, function(x){
    NormalizeData(x, assay = 'ADT', normalization.method = 'CLR', margin = 2)})

In [ ]:
df_build <- function(seurat_object){
    meta <- seurat_object@meta.data
    meta$TCRValpha7.2 <- as.vector(seurat_object@assays$ADT['TCR-Valpha7.2',])
    meta$CD161 <- as.vector(seurat_object@assays$ADT['CD161',])
    meta$CD197 <- as.vector(seurat_object@assays$ADT['CD197',])
    meta$CD45RA <- as.vector(seurat_object@assays$ADT['CD45RA',])
    meta$CD27 <- as.vector(seurat_object@assays$ADT['CD27',])
    meta$imputed_CD197 <- as.vector(seurat_object@assays$x10_ADT['CD197',])
    meta$raw_CD197 <- as.vector(seurat_object@assays$ADT@counts['CD197',])
    meta$raw_CD45RA <- as.vector(seurat_object@assays$ADT@counts['CD45RA',])
    meta$raw_CD27 <- as.vector(seurat_object@assays$ADT@counts['CD27',])
    return(meta)}

# Naive vs Non-Naive

In [ ]:
nonmait_b065 <- cd8.batch$`B065`
nonmait_b069 <- cd8.batch$`B069`
nonmait_b076 <- cd8.batch$`B076`

In [ ]:
nonmait_b065_df <- df_build(nonmait_b065)
nonmait_b069_df <- df_build(nonmait_b069)
nonmait_b076_df <- df_build(nonmait_b076)

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
p1 <- ggplot(nonmait_b065_df, aes(x = CD197, y = CD45RA, color = predicted.t_celltype.l2)) + 
stat_density_2d(geom = 'polygon', aes(alpha = ..level.., fill = predicted.t_celltype.l2), bins = 10) +
geom_vline(xintercept = 1.1) & geom_hline(yintercept = 3) & ggtitle('B065')

p2 <- ggplot(nonmait_b069_df, aes(x = CD197, y = CD45RA, color = predicted.t_celltype.l2)) + 
stat_density_2d(geom = 'polygon', aes(alpha = ..level.., fill = predicted.t_celltype.l2), bins = 10) +
geom_vline(xintercept = 1.1) & geom_hline(yintercept = 3.1) & ggtitle('B069')

p3 <- ggplot(nonmait_b076_df, aes(x = CD197, y = CD45RA, color = predicted.t_celltype.l2)) + 
stat_density_2d(geom = 'polygon', aes(alpha = ..level.., fill = predicted.t_celltype.l2), bins = 10) +
geom_vline(xintercept = 1) & geom_hline(yintercept = 3.45) & ggtitle('B076')

p1 + p2 + p3

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
p1 <- ggplot(nonmait_b065_df, aes(x = CD27, y = CD45RA, color = predicted.t_celltype.l2)) + 
stat_density_2d(geom = 'polygon', aes(alpha = ..level.., fill = predicted.t_celltype.l2), bins = 10) +
geom_vline(xintercept = 2.8) & geom_hline(yintercept = 3) & ggtitle('B065')

p2 <- ggplot(nonmait_b069_df, aes(x = CD27, y = CD45RA, color = predicted.t_celltype.l2)) + 
stat_density_2d(geom = 'polygon', aes(alpha = ..level.., fill = predicted.t_celltype.l2), bins = 10) +
geom_vline(xintercept = 2.5) & geom_hline(yintercept = 3.1) & ggtitle('B069')

p3 <- ggplot(nonmait_b076_df, aes(x = CD27, y = CD45RA, color = predicted.t_celltype.l2)) + 
stat_density_2d(geom = 'polygon', aes(alpha = ..level.., fill = predicted.t_celltype.l2), bins = 10) +
geom_vline(xintercept = 2.4) & geom_hline(yintercept = 3.45) & ggtitle('B076')

p1 + p2 + p3

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
p1 <- FeatureScatter(object = nonmait_b065, feature1 = 'adt_CD197', feature2 = 'adt_CD45RA', group.by = 'predicted.t_celltype.l2', pt.size = 0.0001) &
geom_vline(xintercept = 1.1) & geom_hline(yintercept = 3) & ggtitle('B065')
p2 <- FeatureScatter(object = nonmait_b069, feature1 = 'adt_CD197', feature2 = 'adt_CD45RA', group.by = 'predicted.t_celltype.l2', pt.size = 0.0001) & 
geom_vline(xintercept = 1.1) & geom_hline(yintercept = 3.1) & ggtitle('B069')
p3 <- FeatureScatter(object = nonmait_b076, feature1 = 'adt_CD197', feature2 = 'adt_CD45RA', group.by = 'predicted.t_celltype.l2', pt.size = 0.0001) & 
geom_vline(xintercept = 1) & geom_hline(yintercept = 3.45) & ggtitle('B076')
p4 <- FeatureScatter(object = nonmait_b065, feature1 = 'x10ADT_CD197', feature2 = 'adt_CD45RA', group.by = 'predicted.t_celltype.l2', pt.size = 0.0001) &
geom_vline(xintercept = 0.6) & geom_hline(yintercept = 3) & ggtitle('B065')
p5 <- FeatureScatter(object = nonmait_b069, feature1 = 'x10ADT_CD197', feature2 = 'adt_CD45RA', group.by = 'predicted.t_celltype.l2', pt.size = 0.0001) &
geom_vline(xintercept = 0.6) & geom_hline(yintercept = 3) & ggtitle('B069')
p6 <- FeatureScatter(object = nonmait_b076, feature1 = 'x10ADT_CD197', feature2 = 'adt_CD45RA', group.by = 'predicted.t_celltype.l2', pt.size = 0.0001) &
geom_vline(xintercept = 0.6) & geom_hline(yintercept = 3.45) & ggtitle('B076')

p1 + p2 + p3
p4 + p5 + p6

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
p1 <- FeatureScatter(object = nonmait_b065, feature1 = 'adt_CD27', feature2 = 'adt_CD45RA', group.by = 'predicted.t_celltype.l2', pt.size = 0.0001) & 
geom_vline(xintercept = 2.8) & geom_hline(yintercept = 3) & ggtitle('B065')
p2 <- FeatureScatter(object = nonmait_b069, feature1 = 'adt_CD27', feature2 = 'adt_CD45RA', group.by = 'predicted.t_celltype.l2', pt.size = 0.0001) & 
geom_vline(xintercept = 2.5) & geom_hline(yintercept = 3.1) & ggtitle('B069')
p3 <- FeatureScatter(object = nonmait_b076, feature1 = 'adt_CD27', feature2 = 'adt_CD45RA', group.by = 'predicted.t_celltype.l2', pt.size = 0.0001) & 
geom_vline(xintercept = 2.4) & geom_hline(yintercept = 3.45) & ggtitle('B076')

p1 + p2 + p3

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
p1 <- FeatureScatter(object = nonmait_b065, feature1 = 'adt_CD197', feature2 = 'adt_CD45RA', group.by = 'batch_id',cols = 'black', pt.size = 0.0001) & geom_density_2d(size = 1) &
geom_vline(xintercept = 1.1) & geom_hline(yintercept = 3) & ggtitle('B065')
p2 <- FeatureScatter(object = nonmait_b069, feature1 = 'adt_CD197', feature2 = 'adt_CD45RA', group.by = 'batch_id',cols = 'black', pt.size = 0.0001) & geom_density_2d(size = 1) &
geom_vline(xintercept = 1.1) & geom_hline(yintercept = 3.1) & ggtitle('B069')
p3 <- FeatureScatter(object = nonmait_b076, feature1 = 'adt_CD197', feature2 = 'adt_CD45RA', group.by = 'batch_id',cols = 'black', pt.size = 0.0001) & geom_density_2d(size = 1) &
geom_vline(xintercept = 1) & geom_hline(yintercept = 3.45) & ggtitle('B076')

p1 + p2 + p3

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
p1 <- FeatureScatter(object = nonmait_b065, feature1 = 'adt_CD27', feature2 = 'adt_CD45RA', group.by = 'batch_id',cols = 'black', pt.size = 0.0001) & geom_density_2d(size = 1) &
geom_vline(xintercept = 2.8) & geom_hline(yintercept = 3) & ggtitle('B065')
p2 <- FeatureScatter(object = nonmait_b069, feature1 = 'adt_CD27', feature2 = 'adt_CD45RA', group.by = 'batch_id',cols = 'black', pt.size = 0.0001) & geom_density_2d(size = 1) &
geom_vline(xintercept = 2.5) & geom_hline(yintercept = 3.1) & ggtitle('B069')
p3 <- FeatureScatter(object = nonmait_b076, feature1 = 'adt_CD27', feature2 = 'adt_CD45RA', group.by = 'batch_id',cols = 'black', pt.size = 0.0001) & geom_density_2d(size = 1) &
geom_vline(xintercept = 2.4) & geom_hline(yintercept = 3.45) & ggtitle('B076')

p1 + p2 + p3

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
p1 <- ggplot(nonmait_b065_df, aes(x = CD45RA)) + geom_histogram(binwidth = .01) + geom_vline(xintercept = 3) & ggtitle('B065')

p2 <- ggplot(nonmait_b065_df, aes(x = CD45RA)) + geom_histogram(binwidth = .01) + geom_vline(xintercept = 3.1) & ggtitle('B069')

p3 <- ggplot(nonmait_b065_df, aes(x = CD45RA)) + geom_histogram(binwidth = .01) + geom_vline(xintercept = 3.45) & ggtitle('B076')

p1 + p2 + p3

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
p1 <- ggplot(nonmait_b065_df, aes(x = raw_CD45RA)) + geom_histogram() & ggtitle('B065')

p2 <- ggplot(nonmait_b065_df, aes(x = raw_CD45RA)) + geom_histogram() & ggtitle('B069')

p3 <- ggplot(nonmait_b065_df, aes(x = raw_CD45RA)) + geom_histogram() & ggtitle('B076')

p1 + p2 + p3

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
p1 <- ggplot(nonmait_b065_df, aes(x = CD197)) + geom_histogram(binwidth = .01) + geom_vline(xintercept = 1.1) & ggtitle('B065')

p2 <- ggplot(nonmait_b065_df, aes(x = CD197)) + geom_histogram(binwidth = .01) + geom_vline(xintercept = 1.1) & ggtitle('B069')

p3 <- ggplot(nonmait_b065_df, aes(x = CD197)) + geom_histogram(binwidth = .01) + geom_vline(xintercept = 1) & ggtitle('B076')

p1 + p2 + p3

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
p1 <- ggplot(nonmait_b065_df, aes(x = raw_CD197)) + geom_histogram() & ggtitle('B065')

p2 <- ggplot(nonmait_b065_df, aes(x = raw_CD197)) + geom_histogram() & ggtitle('B069')

p3 <- ggplot(nonmait_b065_df, aes(x = raw_CD197)) + geom_histogram() & ggtitle('B076')

p1 + p2 + p3

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
p1 <- ggplot(nonmait_b065_df, aes(x = imputed_CD197)) + geom_histogram(binwidth = .01) & ggtitle('B065')

p2 <- ggplot(nonmait_b065_df, aes(x = imputed_CD197)) + geom_histogram(binwidth = .01) & ggtitle('B069')

p3 <- ggplot(nonmait_b065_df, aes(x = imputed_CD197)) + geom_histogram(binwidth = .01) & ggtitle('B076')

p1 + p2 + p3

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
p1 <- ggplot(nonmait_b065_df, aes(x = CD27)) + geom_histogram(binwidth = .01) + geom_vline(xintercept = 2.8) & ggtitle('B065')

p2 <- ggplot(nonmait_b065_df, aes(x = CD27)) + geom_histogram(binwidth = .01) + geom_vline(xintercept = 2.5) & ggtitle('B069')

p3 <- ggplot(nonmait_b065_df, aes(x = CD27)) + geom_histogram(binwidth = .01) + geom_vline(xintercept = 2.4) & ggtitle('B076')

p1 + p2 + p3

In [ ]:
naive_b065 <- subset(nonmait_b065, subset = adt_CD197 > 1.1 & adt_CD45RA > 3 & adt_CD27 > 2.8)
naive_b069 <- subset(nonmait_b069, subset = adt_CD197 > 1.1 & adt_CD45RA > 3.1 & adt_CD27 > 2.5)
naive_b076 <- subset(nonmait_b076, subset = adt_CD197 > 1 & adt_CD45RA > 3.45 & adt_CD27 > 2.4)

## Naive Subset Combined

In [ ]:
naive_merge <- merge(naive_b065, c(naive_b069, naive_b076))

In [ ]:
table(naive_merge$pbmc_sample_id)

In [ ]:
naive_merge$gating_celltype <- rep("CD8 Naive", length(colnames(naive_merge[['RNA']])))

In [ ]:
saveRDS(naive_merge, file = '../Seurat_Objects/cd8_naive.rds')

# Memory Analysis

In [ ]:
mem_b065 <- subset(nonmait_b065, subset = adt_CD197 > 1.1 & adt_CD45RA > 3 & adt_CD27 > 2.8, invert =T)
mem_b065
mem_b069 <- subset(nonmait_b069, subset = adt_CD197 > 1.1 & adt_CD45RA > 3.1 & adt_CD27 > 2.5, invert =T)
mem_b069
mem_b076 <- subset(nonmait_b076, subset = adt_CD197 > 1 & adt_CD45RA > 3.45 & adt_CD27 > 2.4, invert =T)
mem_b076

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
p1 <- FeatureScatter(object = mem_b065, feature1 = 'adt_CD197', feature2 = 'adt_CD45RA', group.by = 'predicted.t_celltype.l2', pt.size = 0.0001) &
geom_vline(xintercept = 1.4) & geom_hline(yintercept = 3) & ggtitle('B065')
p2 <- FeatureScatter(object = mem_b069, feature1 = 'adt_CD197', feature2 = 'adt_CD45RA', group.by = 'predicted.t_celltype.l2', pt.size = 0.0001) &
geom_vline(xintercept = 1.5) & geom_hline(yintercept = 3.1) & ggtitle('B069')
p3 <- FeatureScatter(object = mem_b076, feature1 = 'adt_CD197', feature2 = 'adt_CD45RA', group.by = 'predicted.t_celltype.l2', pt.size = 0.0001) &
geom_vline(xintercept = 1.5) & geom_hline(yintercept = 3.45) & ggtitle('B076')
p1 + p2 + p3

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
p1 <- FeatureScatter(object = mem_b065, feature1 = 'adt_CD27', feature2 = 'adt_CD45RA', group.by = 'predicted.t_celltype.l2', pt.size = 0.0001) &
geom_vline(xintercept = 2.8) & geom_hline(yintercept = 3) & ggtitle('B065')
p2 <- FeatureScatter(object = mem_b069, feature1 = 'adt_CD27', feature2 = 'adt_CD45RA', group.by = 'predicted.t_celltype.l2', pt.size = 0.0001) &
geom_vline(xintercept = 2.5) & geom_hline(yintercept = 3.1) & ggtitle('B069')
p3 <- FeatureScatter(object = mem_b076, feature1 = 'adt_CD27', feature2 = 'adt_CD45RA', group.by = 'predicted.t_celltype.l2', pt.size = 0.0001) &
geom_vline(xintercept = 2.4) & geom_hline(yintercept = 3.45) & ggtitle('B076')

p1 + p2 + p3

## CM Subset

In [ ]:
cm_b065 <- subset(mem_b065, subset = adt_CD27 > 2.8 & adt_CD45RA < 3 & adt_CD197 > 1.4)
cm_b069 <- subset(mem_b069, subset = adt_CD27 > 2.5 & adt_CD45RA < 3.1 & adt_CD197 > 1.5)
cm_b076 <- subset(mem_b076, subset = adt_CD27 > 2.4 & adt_CD45RA < 3.45 & adt_CD197 > 1.5)

In [ ]:
cm_merge <- merge(cm_b065, c(cm_b069, cm_b076))

In [ ]:
table(cm_merge$pbmc_sample_id)

In [ ]:
cm_merge$gating_celltype <- rep("CD8 CM", length(colnames(cm_merge[['RNA']])))

In [ ]:
saveRDS(cm_merge, file = '../Seurat_Objects/cd8_cm.rds')

## EM Subset

In [ ]:
em1_b065 <- subset(mem_b065, subset = adt_CD27 > 2.8 & adt_CD45RA < 3 & adt_CD197 < 1.4)
em1_b069 <- subset(mem_b069, subset = adt_CD27 > 2.5 & adt_CD45RA < 3.1 & adt_CD197 < 1.5)
em1_b076 <- subset(mem_b076, subset = adt_CD27 > 2.4 & adt_CD45RA < 3.45 & adt_CD197 < 1.5)

In [ ]:
em2_b065 <- subset(mem_b065, subset = adt_CD27 < 2.8 & adt_CD45RA < 3 & adt_CD197 < 1.4)
em2_b069 <- subset(mem_b069, subset = adt_CD27 < 2.5 & adt_CD45RA < 3.1 & adt_CD197 < 1.5)
em2_b076 <- subset(mem_b076, subset = adt_CD27 < 2.4 & adt_CD45RA < 3.45 & adt_CD197 < 1.5)

In [ ]:
em1_merge <- merge(em1_b065, c(em1_b069, em1_b076))
em2_merge <- merge(em2_b065, c(em2_b069, em2_b076))

In [ ]:
table(em1_merge$pbmc_sample_id)
table(em2_merge$pbmc_sample_id)

In [ ]:
em1_merge$gating_celltype <- rep("CD8 EM1", length(colnames(em1_merge[['RNA']])))
em2_merge$gating_celltype <- rep("CD8 EM2", length(colnames(em2_merge[['RNA']])))

In [ ]:
em_merge <- merge(em1_merge, em2_merge)

In [ ]:
saveRDS(em_merge, file = '../Seurat_Objects/cd8_em.rds')

## EMRA Subset

In [ ]:
emra_b065 <- subset(mem_b065, subset = adt_CD27 < 2.8 & adt_CD45RA > 3 & adt_CD197 < 1.4)
emra_b069 <- subset(mem_b069, subset = adt_CD27 < 2.5 & adt_CD45RA > 3.1 & adt_CD197 < 1.5)
emra_b076 <- subset(mem_b076, subset = adt_CD27 < 2.4 & adt_CD45RA > 3.45 & adt_CD197 < 1.5)

In [ ]:
emra_merge <- merge(emra_b065, c(emra_b069, emra_b076))

In [ ]:
table(emra_merge$pbmc_sample_id)

In [ ]:
emra_merge$gating_celltype <- rep("CD8 TEMRA", length(colnames(emra_merge[['RNA']])))

In [ ]:
saveRDS(emra_merge, file = '../Seurat_Objects/cd8_emra.rds')

# Recombine

In [ ]:
merge_all <- merge(naive_merge, c(cm_merge, em_merge, emra_merge))
merge_all

In [ ]:
table(merge_all$gating_celltype)

In [ ]:
merge_all.split <- SplitObject(merge_all, split.by = 'gating_celltype')

In [ ]:
lapply(merge_all.split, function(x){
    table(x$pbmc_sample_id)})

## Subset Remaining cells

In [ ]:
bcs <- setdiff(colnames(cd8[['RNA']]), colnames(merge_all[['RNA']]))

In [ ]:
unknowns <- subset(cd8, cells = bcs)

In [ ]:
unknowns$gating_celltype <- rep("CD8 Unk", length(colnames(unknowns[['RNA']])))

In [ ]:
saveRDS(unknowns, file = '../Seurat_Objects/cd8_unk.rds')

In [ ]:
merge_all_final <- merge(merge_all, unknowns)
merge_all_final

## Major Populations Plots

In [ ]:
main_subset <- merge(naive_merge, c(cm_merge, em_merge, emra_merge))

In [ ]:
main.batch <- SplitObject(main_subset, 'batch_id')

In [ ]:
df_main_b065 <- df_build(main.batch$`B065`)
df_main_b069 <- df_build(main.batch$`B069`)
df_main_b076 <- df_build(main.batch$`B076`)

In [ ]:
p1 <- ggplot(df_main_b065, aes(x = CD27, y = CD45RA, color = gating_celltype)) + 
stat_density_2d(geom = 'polygon', aes(alpha = ..level.., fill = gating_celltype), bins = 10)
p2 <- ggplot(df_main_b065, aes(x = CD197, y = CD45RA, color = gating_celltype)) + 
stat_density_2d(geom = 'polygon', aes(alpha = ..level.., fill = gating_celltype), bins = 10)

In [ ]:
ggplot(df_main_b069, aes(x = CD27, y = CD45RA, color = gating_celltype)) + 
stat_density_2d(geom = 'polygon', aes(alpha = ..level.., fill = gating_celltype), bins = 10)
ggplot(df_main_b069, aes(x = CD197, y = CD45RA, color = gating_celltype)) + 
stat_density_2d(geom = 'polygon', aes(alpha = ..level.., fill = gating_celltype), bins = 10)

In [ ]:
ggplot(df_main_b076, aes(x = CD27, y = CD45RA, color = gating_celltype)) + 
stat_density_2d(geom = 'polygon', aes(alpha = ..level.., fill = gating_celltype), bins = 10)
ggplot(df_main_b076, aes(x = CD197, y = CD45RA, color = gating_celltype)) + 
stat_density_2d(geom = 'polygon', aes(alpha = ..level.., fill = gating_celltype), bins = 10)

# Get Counts

In [ ]:
merge_all.split <- SplitObject(merge_all, 'gating_celltype')

In [ ]:
lapply(merge_all.split, function(x){
    table(x$pbmc_sample_id)})